<h2>Desarrollo del Desafio 2 - Machine Learning


<h3>Grupo 7

<li>Laura Jazmín Chao
<li>Juan Nicolás Capistrano
<li>Betiana Srur
<li>Ignacio Mendieta
<li>Marecelo Carrizo

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [72]:
import numpy as np
import pandas as pd

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Libreria para analisis ML
from sklearn import datasets, linear_model

In [73]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
88489,88489,sell,apartment,San Telmo,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,3428113.0,"-34.6235141,-58.376459",-34.623514,...,48.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1b7df_venta_depart...,Formando Parte del Barrio de San Telmo: Caract...,Departamento - 2 Ambientes - San Telmo,https://thumbs4.properati.com/0/77E073-VV6svLs...
26829,26829,sell,apartment,Castelar,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Cast...,Argentina,Bs.As. G.B.A. Zona Oeste,3435607.0,"-34.6506667,-58.6139125",-34.650667,...,50.0,2100.000000,2100.00000,NaN,2.0,NaN,http://www.properati.com.ar/17rk0_venta_depart...,Actualizado hace 2 segundosDepartamentos plant...,Venta departamento Morn,https://thumbs4.properati.com/9/Jt7FCEZOAQ3q4l...
6474,6474,sell,apartment,La Lucila,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3432310.0,"-34.4930867289,-58.4846887498",-34.493087,...,184.0,3963.414634,4415.76087,NaN,4.0,20200.0,http://www.properati.com.ar/15ug4_venta_depart...,"Unidad de 4 ambientes, pasante del frente al c...",Carpe Diem 4 ambientes c/ Dependencia y Cocher...,https://thumbs4.properati.com/0/Evwf1kVj6fSb7P...
89811,89811,sell,apartment,San Telmo,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,3428113.0,"-34.6251347,-58.3789013",-34.625135,...,35.0,1715.000000,1960.00000,NaN,NaN,NaN,http://www.properati.com.ar/1b95y_venta_depart...,Bernardo de Irigoyen 1418: Ubicada en pleno co...,Departamento - San Telmo,https://thumbs4.properati.com/6/FuOVP_alpj9HA0...
108025,108025,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.5888889,-58.4305556",-34.588889,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1bza9_venta_depart...,CODIGO: 916-69723 ubicado en: Scalabrini Ortiz...,"Departamento en Venta en Palermo, Capital Federal",https://thumbs4.properati.com/4/tNhhA8LKl8JILQ...


In [74]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Limpieza de Datos.

In [75]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
# Observamos esos registros
data.loc[currency_OTHER_CURRENCY_mask, :]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [76]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data.shape[0]
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data.shape[0]}')

Cantidad de registros original - registro actuales: 3


In [77]:
# Creamos la lista de columnas a filtrar
drop_columns = ['properati_url', 'Unnamed: 0', 'operation','image_thumbnail','expenses','lat-lon','geonames_id','lat', 'lon']

# Creamos una variable nueva para guardar los datos (el cambio se realiza inplace=False)
data_filtered = data.drop(drop_columns, axis=1)
data_filtered.head(5)

,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


**Esto no es necesario ahora ¿?**

In [78]:
# Creamos un diccionario con la nueva agrupación
provincia={'Capital Federal':'CABA','Bs.As. G.B.A. Zona Norte': 'Bs. As. G.B.A.', 'Bs.As. G.B.A. Zona Sur':'Bs. As. G.B.A.', 'Córdoba':'Córdoba','Santa Fe':'Santa Fe','Buenos Aires Costa Atlántica':'Buenos Aires', 'Bs.As. G.B.A. Zona Oeste':'Bs. As. G.B.A.', 'Buenos Aires Interior':'Buenos Aires','Río Negro':'Resto del país','Neuquén':'Resto del país', 'Mendoza':'Resto del país', 'Tucumán':'Resto del país', 'Corrientes':'Resto del país', 'Misiones':'Resto del país', 'Entre Ríos':'Resto del país', 'Salta':'Resto del país', 'Chubut':'Resto del país', 'San Luis':'Resto del país', 'La Pampa':'Resto del país','Formosa':'Resto del país','Chaco': 'Resto del país', 'San Juan':'Resto del país','Tierra Del Fuego':'Resto del país', 'Catamarca':'Resto del país', 'Jujuy': 'Resto del país', 'Santa Cruz':'Resto del país', 'La Rioja':'Resto del país', 'Santiago Del Estero':'Resto del país'}

In [79]:
# Mapeamos las provincias con el diccionario y ponermos el grupo en una columna nueva
data_filtered['province_group'] = data_filtered.state_name.map(provincia)
display(data_filtered.sample(5))


,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title,province_group
81856,house,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,350000.0,USD,6175575.00,350000.0,326.0,326.0,1073.619632,1073.619632,NaN,NaN,Muy linda casa desarrollada en dos plantas de ...,Casa en Los Senderos I,Bs. As. G.B.A.
19130,apartment,Congreso,|Argentina|Capital Federal|Congreso|,Argentina,Capital Federal,72500.0,USD,1279226.25,72500.0,28.0,28.0,2589.285714,2589.285714,NaN,1.0,Elegante y moderno edificio de aspecto clásico...,Monoambiente en venta Congreso a estrenar c/am...,CABA
121145,house,Banfield,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,290000.0,USD,5116905.00,290000.0,170.0,170.0,1705.882353,1705.882353,NaN,NaN,"Casa sobre linea municipal, estar de recepción...",Casa - Banfield Oeste,Bs. As. G.B.A.
53310,store,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Local 36 m2. en Edificio en construcción. Sani...,SAN LUIS 2300,Santa Fe
114679,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEPARTAMENTO DE 3 DORMITORIOS en TORRES DEL RI...,"ALBERDI, TORRES DEL RIO (dpto.3dor.)",Córdoba


<h2>Tomamos solamente CABA para el analisis.

In [80]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data_filtered.state_name == 'Capital Federal' 
data_caba = data_filtered.loc[data_caba_mask, :]

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['province_group', 'country_name', 'state_name', 'place_with_parent_names','description', 'title'], inplace=True)

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Vemos la cantidad de datos solamente de CABA.
data_caba.shape

(32314, 12)

In [82]:
# Revisamos si hay barrios de caba mal indentificados "Ejemplo: Capital Federal"
pd.set_option('display.max_rows', 1000) # Para mostrar todas las filas.


In [83]:
# Dropeamos los de Capital Federal
capi_mask = data_caba.place_name == 'Capital Federal'
data_caba.drop(data_caba.loc[capi_mask, :].index, inplace=True)

In [84]:
data_caba.shape

(31019, 12)

In [85]:
data_caba.head(20)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,NaN,1.0
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,NaN,1.0
16,PH,Mataderos,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,NaN,4.0
19,apartment,Palermo,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0
21,apartment,Palermo,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,NaN,4.0


## De los datos filtrados según Desafio 1 deberiamos:

1. Terminar de dropear variables incompletas que serian necesarias para poder entrenar un modelo. (ejemplo: si la propiedad no posee: price, price_aprox_local_currency,price_usd_per_m2, price_per_m2, etc...) la información de los barrios ya fue analizado y dropeado en el paso anterior.
2. Definir las variables CUANTITATIVAS, ejemplo precio, total de m2 de superficie.
3. Armar las variables CUALITATICAS, Dummy (ejemplo: place_name, property_type).
4. Armar una nueva variable Dummy que contemple los precios de propiedad segun su distribución por metro cuadrado. Utilizando Panda CUT. 

## Finalizado el analisis de datos, armamos diferentes modelos para predecir el precio de la propiedad.

1. Regresion Lineal Multiple
2. Entrenar los modelos con Lasso, RidgeCV y utilizando Cross Validation.



### Primero damos homogenidad a los datos

Vamos a realizar ciertas verificaciones de los datos.

In [86]:
# 1. Verificamos que el price y el price_aprox_usd sean los mismos, y luego si fuera así dropeamos el price_aprox_usd

mask_price_validation = data_caba.price != data_caba.price_aprox_usd

display(data_caba.loc[mask_price_validation, :].shape)

# Detectamos que hay 4138 propiedades donde los valores son diferentes? pero estan en Pesos o son NaN?

display(data_caba.loc[mask_price_validation, :].sample(20))

# Detectamos que tenemos los dos casos, donde los precios son NaN y donde la moneda está en ARS (lo cual no estaría mal)
# En el caso de ARS, vemos que el valor de price toma el de la moneda local ~ Por lo que podriamos llegar a eliminar price, currency y price_aprox_local_currency 

# Para los valores de NaN tendriamos que tomar la decisión si los dropeamos o tratamos de calcular un precio promedio en base a la cantidad de metros cuadrados. Podemos hacerlo en base al calculo que realizamos en el Desafio 1 ??

(4138, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
71626,apartment,Palermo Hollywood,NaN,NaN,NaN,NaN,126.0,93.0,NaN,NaN,NaN,NaN
109390,apartment,Caballito,NaN,NaN,NaN,NaN,41.0,36.0,NaN,NaN,NaN,NaN
121009,apartment,Belgrano,NaN,NaN,NaN,NaN,38.0,35.0,NaN,NaN,NaN,NaN
113457,apartment,Palermo,5035125.0,ARS,4982600.71,282388.32,118.0,75.0,2393.121356,67135.0,NaN,2.0
9327,apartment,Once,NaN,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,NaN
32054,store,Palermo Soho,NaN,NaN,NaN,NaN,120.0,120.0,NaN,NaN,NaN,NaN
88212,store,Balvanera,NaN,NaN,NaN,NaN,1250.0,1250.0,NaN,NaN,NaN,NaN
74891,apartment,Belgrano,NaN,NaN,NaN,NaN,68.0,68.0,NaN,NaN,NaN,NaN
31422,apartment,Villa del Parque,NaN,NaN,NaN,NaN,49.0,44.0,NaN,NaN,NaN,2.0
120805,apartment,Caballito,NaN,NaN,NaN,NaN,85.0,70.0,NaN,NaN,NaN,NaN


In [87]:
# Elimino primero los registros donde no tengo nada de informacion

umbral = 10
data_caba.dropna(axis = 0, thresh=umbral, inplace=True)

# Vuelvo a ver como quedaron los datos
mask_price_validation = data_caba.price != data_caba.price_aprox_usd
display(data_caba.loc[mask_price_validation, :].shape)
display(data_caba.loc[mask_price_validation, :].sample(20))

(1035, 12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
28072,apartment,Palermo,3311974.0,ARS,3277424.76,185747.67,NaN,67.0,NaN,49432.447761,103.0,2.0
64131,apartment,Nuñez,50000000.0,ARS,51462696.12,2916642.36,500.0,105.0,5833.284720,476190.476190,NaN,NaN
86647,apartment,Boca,1960382.0,ARS,1939932.13,109945.43,62.0,56.0,1773.313387,35006.821429,NaN,NaN
48033,apartment,San Telmo,1202900.0,ARS,1190351.78,67463.05,35.0,31.0,1927.515714,38803.225806,NaN,1.0
90160,apartment,Villa Crespo,3354586.0,ARS,3319592.29,188137.51,50.0,42.0,3762.750200,79871.095238,NaN,NaN
51048,apartment,Caballito,994384.0,ARS,984011.00,55768.71,24.0,24.0,2323.696250,41432.666667,NaN,1.0
111133,apartment,Recoleta,1541800.0,ARS,1525716.56,86469.81,34.0,30.0,2543.229706,51393.333333,NaN,1.0
86090,apartment,San Telmo,2243681.0,ARS,2220275.89,125833.88,37.0,33.0,3400.915676,67990.333333,NaN,NaN
105061,apartment,Boedo,1900000.0,ARS,1880179.92,106558.98,71.0,35.0,1500.830704,54285.714286,NaN,NaN
9453,apartment,Caballito,208730.0,ARS,206552.51,11706.34,72.0,66.0,162.588056,3162.575758,7.0,NaN


In [88]:
# Para la superficie de metros cuadadros totales, podemos creer que en caso de ser Nan es porque es un departamente y la cantidad de m2 cubierta sería iguala la cantidad de m2 totales.data_caba


data_caba.surface_total_in_m2.fillna(data_caba.surface_covered_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
59999,apartment,Balvanera,95000.0,USD,1676227.50,95000.0,77.0,77.0,1233.766234,1233.766234,NaN,4.0
63063,apartment,Caballito,119000.0,USD,2099695.50,119000.0,44.0,44.0,NaN,2704.545455,11.0,3.0
29728,apartment,Villa Crespo,140977.0,USD,2487468.67,140977.0,45.0,39.0,3132.822222,3614.794872,9.0,NaN
81639,apartment,Caballito,96000.0,USD,1693872.00,96000.0,41.0,36.0,2341.463415,2666.666667,NaN,NaN
119138,PH,Villa Pueyrredón,112000.0,USD,1976184.00,112000.0,112.0,67.0,1000.000000,1671.641791,NaN,NaN
110894,apartment,San Telmo,139000.0,USD,2452585.50,139000.0,65.0,65.0,NaN,2138.461538,17.0,3.0
24900,apartment,Boedo,430000.0,USD,7587135.00,430000.0,37.0,31.0,11621.621622,13870.967742,NaN,1.0
79695,apartment,Saavedra,176000.0,USD,3105432.00,176000.0,67.0,56.0,2626.865672,3142.857143,NaN,NaN
31201,apartment,Villa Crespo,179000.0,USD,3158365.50,179000.0,95.0,88.0,1884.210526,2034.090909,1.0,4.0
119660,PH,Versalles,220000.0,USD,3881790.00,220000.0,98.0,98.0,NaN,2244.897959,6.0,3.0


In [89]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba.surface_covered_in_m2.fillna(data_caba.surface_total_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
43963,apartment,Belgrano,179000.0,USD,3158365.50,179000.00,63.0,59.0,2841.269841,3033.898305,NaN,3.0
78388,apartment,Flores,71000.0,USD,1252759.50,71000.00,45.0,40.0,1577.777778,1775.000000,NaN,NaN
88070,apartment,Villa Crespo,79285.0,USD,1398944.18,79285.00,47.0,44.0,1686.914894,1801.931818,NaN,NaN
57429,apartment,Caballito,148000.0,USD,2611386.00,148000.00,49.0,49.0,NaN,3020.408163,2.0,2.0
48033,apartment,San Telmo,1202900.0,ARS,1190351.78,67463.05,35.0,31.0,1927.515714,38803.225806,NaN,1.0
110393,apartment,Coghlan,92000.0,USD,1623294.00,92000.00,38.0,38.0,2421.052632,2421.052632,1.0,2.0
81573,apartment,Caballito,102500.0,USD,1808561.25,102500.00,41.0,38.0,2500.000000,2697.368421,NaN,NaN
55751,house,Villa Pueyrredón,130000.0,USD,2293785.00,130000.00,72.0,66.0,1805.555556,1969.696970,NaN,NaN
25588,apartment,Boedo,34200.0,USD,603441.90,34200.00,30.0,30.0,1140.000000,1140.000000,NaN,1.0
61278,apartment,Barrio Norte,121000.0,USD,2134984.50,121000.00,56.0,56.0,2160.714286,2160.714286,2.0,3.0


In [90]:
# Idem al punto anterior pero ahora de surface_total a surface_covered

data_caba.price_usd_per_m2.fillna(data_caba.price_aprox_usd / data_caba.surface_total_in_m2, inplace=True)

data_caba.sample(10)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
9418,apartment,San Telmo,87000.0,USD,1535071.50,87000.0,39.0,39.0,2230.769231,2230.769231,NaN,1.0
77996,apartment,Villa Luro,175000.0,USD,3087787.50,175000.0,61.0,53.0,2868.852459,3301.886792,NaN,NaN
6254,apartment,Coghlan,124000.0,USD,2187918.00,124000.0,55.0,50.0,2254.545455,2480.000000,3.0,NaN
118600,apartment,Palermo,1100000.0,USD,19408950.00,1100000.0,225.0,215.0,4888.888889,5116.279070,NaN,NaN
4321,apartment,Balvanera,145000.0,USD,2558452.50,145000.0,175.0,153.0,828.571429,947.712418,NaN,NaN
75717,apartment,Belgrano,164500.0,USD,2902520.25,164500.0,70.0,66.0,2350.000000,2492.424242,NaN,NaN
62560,apartment,Villa Urquiza,235000.0,USD,4146457.50,235000.0,87.0,87.0,2701.149425,2701.149425,505.0,2.0
88930,apartment,Almagro,71810.0,USD,1267051.54,71810.0,34.0,33.0,2112.058824,2176.060606,NaN,NaN
14861,PH,San Telmo,135000.0,USD,2382007.50,135000.0,89.0,75.0,1516.853933,1800.000000,NaN,3.0
79813,apartment,San Cristobal,72000.0,USD,1270404.00,72000.0,40.0,37.0,1800.000000,1945.945946,NaN,NaN


In [91]:
display(data_caba.floor.count())

# El piso del departamente es un dato muy incompleto por lo que se dropea

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['floor'], inplace=True)

# En el caso de Rooms lo reemplazamos por 0
data_caba.rooms.fillna(0, inplace=True)

2816

In [92]:
display(data_caba.rooms.count())

# En el caso de rooms, podemos tomar que el dato para los NaN es 0?? modifica esto el analisis para predecir? 

23075

In [93]:
# Dropeamos los valores que sabemos que está repetidos o con otra moneda.data_caba

data_caba.drop(columns=['price', 'currency','price_per_m2'], inplace=True)

data_caba.sample(20)

,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms
110779,PH,Flores,2681964.00,152000.0,152.0,81.0,1000.000000,0.0
94180,apartment,Recoleta,5820920.55,329900.0,95.0,88.0,3472.631579,0.0
36242,apartment,Nuñez,9351585.00,530000.0,150.0,150.0,3533.333333,4.0
637,apartment,San Telmo,1658583.00,94000.0,55.0,55.0,1709.090909,3.0
119421,apartment,Coghlan,1711516.50,97000.0,35.0,30.0,2771.428571,1.0
24408,apartment,Villa Crespo,3264232.50,185000.0,73.0,65.0,2534.246575,3.0
43967,apartment,Abasto,1755627.75,99500.0,46.0,46.0,2163.043478,1.0
52645,apartment,Floresta,1676227.50,95000.0,56.0,56.0,1696.428571,3.0
55099,apartment,Palermo Chico,1455671.25,82500.0,25.0,25.0,3300.000000,1.0
77325,apartment,Villa Crespo,1411560.00,80000.0,32.0,28.0,2500.000000,0.0


In [94]:
print('\nPorcentaje de valores incompletos por columna:')
print(f'{round(100 * data_caba.isnull().sum()/data_caba.shape[0], 2)}')


Porcentaje de valores incompletos por columna:
property_type                 0.0
place_name                    0.0
price_aprox_local_currency    0.0
price_aprox_usd               0.0
surface_total_in_m2           0.0
surface_covered_in_m2         0.0
price_usd_per_m2              0.0
rooms                         0.0
dtype: float64


### Falta reindexar las filas.

Dummies de property_type y place_name, no se si conviene realizar el trabajo de la extracción de rooms, hay que evaluar el esfuerzo.

